In [426]:
pip install galois

Note: you may need to restart the kernel to use updated packages.


In [427]:
import galois
import numpy as np

In [428]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(2**3)

In [429]:
#definisco una matrice k x (n-k) dove n-k = 4
k = 3 #le righe
n_k = 4 #le colonne

In [430]:
A = field.Random((k, n_k))
print(A)

[[4 1 7 5]
 [2 4 6 1]
 [5 0 5 1]]


In [431]:
matrice_bin1 = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [0, 1, 0]
])

matrice_bin1 = field(matrice_bin1)

matrice_bin2 = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 1, 0, 0]
])

matrice_bin2 = field(matrice_bin2)

In [432]:
A_primo_intermedio = field(np.dot(matrice_bin1, A))
print(A_primo_intermedio)

[[5 0 5 1]
 [4 1 7 5]
 [2 4 6 1]]


In [433]:
A_primo = field(np.dot(A_primo_intermedio, matrice_bin2))
print(A_primo)

[[5 1 0 5]
 [4 5 1 7]
 [2 1 4 6]]


In [434]:
import threading

In [435]:
#qui in accordo con slide 18 articolo devo ordinare la matrice A e A_primo per righe e colonne, ordinamento per riga 
#è dato dalla somma delle righe in ordine decrescente, mentre per colonna in ordine numerico del primo elemento e in caso di 
#parità del secondo e così via

#lista per conservare somme delle righe, inizializzata con gli zeri
somme_rig = [field(0)] * A_primo.shape[0]
primi_colonne = [0] * A_primo.shape[1]
secondi_colonne = [0] * A_primo.shape[1]

#creo il lock per evitare conglitti nell'aggiornamento
lock = threading.Lock()

#funzione per somma elementi nella colonna
def somma_riga(matrice, indice_riga):
    riga = matrice[indice_riga, :]
    print("Primo print: ", riga) #questi vanno eliminati poi
    print("Secondo print: ", type(riga))
    somma = np.sum(riga)
    print("Terzo print: ", somma) #questi vanno eliminati poi
    print("Quarto print: ",type(somma))
    with lock: #vado ad usare il lock
        somme_rig[indice_riga] = somma
        print(somme_rig)

def estrai_da_colonna(matrice, indice_colonna):
    colonna  = matrice[:, indice_colonna]
    with lock:
        primi_colonne[indice_colonna] = colonna[0]
        secondi_colonne[indice_colonna] = colonna[1]


In [436]:
threads_righe = []
threds_colonne = []
colonne_finali = []

In [437]:
#creo un thread per ogni colonna
for indice_riga in range(A_primo.shape[0]):
    t = threading.Thread(target = somma_riga, args = (A_primo, indice_riga))
    print(t)
    threads_righe.append(t)
    t.start() #avvio il thread

#aspetto il risultato di tutti i threads
for t in threads_righe:
    t.join()


<Thread(Thread-244 (somma_riga), initial)>
Primo print:  [5 1 0 5]
Secondo print:  <class 'galois.GF(2^3)'>
<Thread(Thread-245 (somma_riga), initial)>
Terzo print:  [5 1 0 5]
Quarto print:  <class 'galois.GF(2^3)'>
[GF([5, 1, 0, 5], order=2^3), GF(0, order=2^3), GF(0, order=2^3)]
Primo print:  [4 5 1 7]
Secondo print:  <class 'galois.GF(2^3)'>
Terzo print:  [4 5 1 7]
Quarto print:  <class 'galois.GF(2^3)'>
[GF([5, 1, 0, 5], order=2^3), GF([4, 5, 1, 7], order=2^3), GF(0, order=2^3)]
<Thread(Thread-246 (somma_riga), initial)>
Primo print:  [2 1 4 6]
Secondo print:  <class 'galois.GF(2^3)'>
Terzo print:  [2 1 4 6]
Quarto print:  <class 'galois.GF(2^3)'>
[GF([5, 1, 0, 5], order=2^3), GF([4, 5, 1, 7], order=2^3), GF([2, 1, 4, 6], order=2^3)]


In [438]:
print ("\nSomma degli elementi di ciascuna riga: ", somme_rig)
print ("\n Elementi primi estratti: ", primi_colonne)
print ("\n Elementi secondi estratti: ", secondi_colonne)


Somma degli elementi di ciascuna riga:  [GF([5, 1, 0, 5], order=2^3), GF([4, 5, 1, 7], order=2^3), GF([2, 1, 4, 6], order=2^3)]

 Elementi primi estratti:  [0, 0, 0, 0]

 Elementi secondi estratti:  [0, 0, 0, 0]


In [439]:
indici_ordinati = np.argsort(somme_rig)
A_canonica_righe = A_primo[indici_ordinati, :]

print(A_canonica_righe)

IndexError: index 3 is out of bounds for axis 0 with size 3

In [ ]:
for indice_colonna in range(A_primo.shape[1]):
    p = threading.Thread(target = estrai_da_colonna, args = (A_canonica_righe, indice_colonna))
    threds_colonne.append(p)
    p.start()

for p in threds_colonne:
    p.join()

indici_ordinati_col = np.argsort(primi_colonne)


for idx in indici_ordinati_col:
    colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], idx))

colonne_finali.sort(key=lambda x: (x[0], x[1]))

A_CF = A_canonica_righe[:, [col[2] for col in colonne_finali]]

print(A_CF)

[[0 1 5 5]
 [5 6 6 7]
 [4 1 2 4]]
